In [62]:
def _get_table_data(link):
    try:
        data = pd.read_html(link)[0]
        data.columns = data.columns.droplevel()
        return data
    except (ValueError, IndexError):
        return None

def _get_specific_table_data(anchor, keyword):
    links = [l for l in anchor if l and keyword in l]
    if links:
        return _get_table_data(f"https://fbref.com{links[0]}")
    return None

def _process_team_match(team, season, league_id, name):
    team_name = team.split('/')[-1].replace('-Stats', '').replace('-', '_').lower()
    team_mh = pd.read_html(team)[1]
    team_mh['season'] = season
    team_mh['league_id'] = league_id
    team_mh['league_name'] = name
    team_mh['team'] = team_name
    print(f'--{team_name}')

    data = requests.get(team)
    soup = BeautifulSoup(data.text, features='lxml')
    anchor = [link.get("href") for link in soup.find_all('a')]

    for keyword, col_rename in [("all_comps/shooting/", {'Date': 'Date', 'Sh': 'Sh', 'SoT': 'SoT'}),
                                ("all_comps/keeper", {'Date': 'Date', 'Saves': 'Saves'}),
                                ("all_comps/passing", {'Date': 'Date', 'Cmp': 'Cmp', 'Att': 'Att', 'PrgP': 'PrgP', 'KP': 'KP', '1/3': 'pass_3rd'}),
                                ("all_comps/passing_types", {'Date': 'Date', 'Sw': 'Sw', 'Crs': 'Crs'}),
                                ("all_comps/gca", {'Date': 'Date', 'SCA': 'SCA', 'GCA': 'GCA'}),
                                ("all_comps/defense", {'Date': 'Date', 'Tkl': 'Tkl', 'TklW': 'TklW', 'Def 3rd': 'Tkl_Def_3rd', 'Att 3rd': 'Tkl_Att_3rd', 'Blocks': 'Blocks', 'Int': 'Int'}),
                                ("all_comps/possession", {'Date': 'Date', 'Att 3rd': 'Touches_Att_3rd'}),
                                ("all_comps/misc", {'Date': 'Date', 'Fls': 'Fls', 'Off': 'Off', 'Recov': 'Recov'})]:
        table_data = _get_specific_table_data(anchor, keyword)
        if table_data is not None:
            team_mh = team_mh.merge(table_data, on='Date')
            team_mh.rename(columns=col_rename, inplace=True)

    return team_mh

def _get_team_match_history(self, season, urls):
    match_history = []
    for team in urls:
        team_mh = _process_team_match(team, season, self.league_id, self.name)
        match_history.append(team_mh)
        time.sleep(10)
    match_history = pd.concat(match_history)
    return match_history
